# Polycam lidar data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.cluster
import sklearn.preprocessing
import trimesh
from sklearn.cluster import KMeans

In [2]:
import open3d as o3d

In [3]:
#changing the name of the ply file for different data
pc_ply=o3d.io.read_point_cloud("standing.ply")

### Visualization:

In [ ]:
o3d.visualization.draw_geometries([pc_ply])

### Voxel Downsampling:

In [4]:
downpcd=pc_ply.voxel_down_sample(voxel_size=0.1)


In [ ]:
o3d.visualization.draw_geometries([downpcd])

### Surface reconstruction:

In [ ]:
import matplotlib.pyplot as plt
import os
import sys

In [ ]:
alpha=0.07
print(f"alpha={alpha:.3f}")
mesh=o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pc_ply,alpha)
o3d.visualization.draw_geometries([mesh],mesh_show_back_face=True)

### Outliers:

In [ ]:
#this function finds the outliers in the data and visualizes the point cloud data by showing 
#the 
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

### Statistical Outliers:

In [ ]:
cl,ind=downpcd.remove_statistical_outlier(nb_neighbors=20,std_ratio=2.0)
display_inlier_outlier(downpcd,ind)

### Voxel normal estimation:

In [ ]:
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.2,max_nn=50))

In [ ]:
o3d.visualization.draw_geometries([downpcd])

### method2:

In [ ]:
segment_models={}
segments={}
max_plane_idx=20
rest=pc_ply
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=0.02,ransac_n=5,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print("pass",i,"/",max_plane_idx,"done.")

In [ ]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

### method 3:

In [ ]:
segment_models={}
segments={}
max_plane_idx=20
rest=pc_ply
d_threshold=0.02
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=0.02,ransac_n=5,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    labels = np.array(segments[i].cluster_dbscan(eps=d_threshold*10, min_points=10))
    print(labels)
    candidates=[len(np.where(labels==j)[0]) for j in np.unique(labels)]
    best_candidate=int(np.unique(labels)[np.where(candidates==np.max(candidates))[0]])
    print("the best candidate is: ", best_candidate)
    rest = rest.select_by_index(inliers, invert=True)+segments[i].select_by_index(list(np.where(labels!=best_candidate)[0]))
    segments[i]=segments[i].select_by_index(list(np.where(labels==best_candidate)[0]))
    segments[i].paint_uniform_color(list(colors[:3]))
    print("pass",i+1,"/",max_plane_idx,"done.")

In [ ]:
labels = np.array(rest.cluster_dbscan(eps=0.05, min_points=5))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab10")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])

# o3d.visualization.draw_geometries([segments.values()])
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])
#o3d.visualization.draw_geometries([segments[i]for i in range(max_plane_idx)]+[rest])
# o3d.visualization.draw_geometries([rest])
# ax=plt.axes(projection='3d')